# Fetal Head Circumference Measurement

In this notebook, we focused on measuring head circumference (HC) using a dataset from the HC18 Grand Challenge. We applied a lightweight deep learning model to perform image segmentation on two-dimensional ultrasound images of fetal heads. This model was designed to achieve both fast and accurate segmentation, which is then used to calculate the fetal HC. Our method provides a reliable and efficient way for obstetricians to measure fetal HC with high accuracy, even in environments with limited computing resources. The evaluation metrics demonstrated that our approach offers precise boundary predictions and high overlap with the ground truth, making it a valuable tool for clinical practice.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import time
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.linear_model import LinearRegression
import shutil
import random

# DATA PREPROCESSING

In [2]:
def img_crop(in_img, crop_size=[512,768]):
    """
    Crop an image to a specific size.
    """

    crop_img = np.zeros([512,768],dtype=in_img.dtype)

    r_out = crop_size[0]
    c_out = crop_size[1]

    r_in, c_in = in_img.shape

    dr = int((r_in - r_out) / 2 + 0.5)
    dc = int((c_in - c_out) / 2 + 0.5)

    if dr > 0:
        rp_in = [dr,dr+r_out]
        rp_out = [0, r_out]
    else:
        dr = -dr
        rp_in = [0,r_in]
        rp_out = [dr,dr+r_in]

    if dc > 0:
        cp_in = [dc,dc+c_out]
        cp_out = [0, c_out]
    else:
        dc = -dc
        cp_in = [0,c_in]
        cp_out = [dc,dc+c_in]

    crop_img[rp_out[0]:rp_out[1],cp_out[0]:cp_out[1]] = in_img[rp_in[0]:rp_in[1],cp_in[0]:cp_in[1]]

    return crop_img

In [3]:
def ellip_fill(in_folder, out_folder):

    dirs = os.listdir(in_folder)
    for i in range(len(dirs)):
        print('Ellip filling: %d / %d' % (i + 1, len(dirs)))
        img_name = dirs[i]
        img_path = in_folder + img_name

        img = cv2.imread(img_path, 0)

        img = 255 - img
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
        ellipse = labels == 2
        ellipse = ellipse * 255
        ellipse = ellipse.astype('uint8')

        # Original label is 3 pixels wide so an expansion process is needed
        kernel = np.ones((3, 3), np.uint8)
        ellipse = cv2.dilate(ellipse, kernel, iterations=1)

        # Save
        save_path = out_folder + img_name
        cv2.imwrite(save_path, ellipse)

In [4]:
def img_augumentation(in_folder, out_folder, data):
    """
    Data augumentation, operations include rotation of 10, 20 and 30 degrees in clockwise and counterclockwise,
    magnification of 1.15 times, narrowing of 0.85 times, left and right flip, up and down flip.
    """

    dirs = os.listdir(in_folder)
    for i in range(len(dirs)):
        print('%s augumentation: %d / %d' % (data, i + 1, len(dirs)))
        img_name = dirs[i]
        img_path = in_folder + img_name
        img = cv2.imread(img_path, 0)
        h,w = img.shape

        # Image scale
        matRotate_085 = cv2.getRotationMatrix2D((w * 0.5, h * 0.5), 0, 0.85)
        matRotate_115 = cv2.getRotationMatrix2D((w * 0.5, h * 0.5), 0, 1.15)

        img_085 = cv2.warpAffine(img, matRotate_085, (w, h))
        img_115 = cv2.warpAffine(img, matRotate_115, (w, h))

        # Flip
        img_Flr = cv2.flip(img, 1)  # flip left-right
        img_Fud = cv2.flip(img, 0)  # flip up-down

        # Rotate clockwise and counterclockwise
        matRotate_p10 = cv2.getRotationMatrix2D((w * 0.5, h * 0.5), 10, 1)
        matRotate_p20 = cv2.getRotationMatrix2D((w * 0.5, h * 0.5), 20, 1)
        matRotate_p30 = cv2.getRotationMatrix2D((w * 0.5, h * 0.5), 30, 1)
        matRotate_m10 = cv2.getRotationMatrix2D((w * 0.5, h * 0.5), -10, 1)
        matRotate_m20 = cv2.getRotationMatrix2D((w * 0.5, h * 0.5), -20, 1)
        matRotate_m30 = cv2.getRotationMatrix2D((w * 0.5, h * 0.5), -30, 1)

        img_Rp10 = cv2.warpAffine(img, matRotate_p10, (w, h))
        img_Rp20 = cv2.warpAffine(img, matRotate_p20, (w, h))
        img_Rp30 = cv2.warpAffine(img, matRotate_p30, (w, h))
        img_Rm10 = cv2.warpAffine(img, matRotate_m10, (w, h))
        img_Rm20 = cv2.warpAffine(img, matRotate_m20, (w, h))
        img_Rm30 = cv2.warpAffine(img, matRotate_m30, (w, h))

        # Image name
        if data=="image":
            img_085_name = img_name[:-4] + '_S.png'
            img_115_name = img_name[:-4] + '_L.png'
            img_Flr_name = img_name[:-4] + '_Flr.png'
            img_Fud_name = img_name[:-4] + '_Fud.png'
            img_Rp10_name = img_name[:-4] + '_Rp10.png'
            img_Rp20_name = img_name[:-4] + '_Rp20.png'
            img_Rp30_name = img_name[:-4] + '_Rp30.png'
            img_Rm10_name = img_name[:-4] + '_Rm10.png'
            img_Rm20_name = img_name[:-4] + '_Rm20.png'
            img_Rm30_name = img_name[:-4] + '_Rm30.png'
        elif data == "label":
            img_085_name = img_name[:-15] + '_S_Annotation.png'
            img_115_name = img_name[:-15] + '_L_Annotation.png'
            img_Flr_name = img_name[:-15] + '_Flr_Annotation.png'
            img_Fud_name = img_name[:-15] + '_Fud_Annotation.png'
            img_Rp10_name = img_name[:-15] + '_Rp10_Annotation.png'
            img_Rp20_name = img_name[:-15] + '_Rp20_Annotation.png'
            img_Rp30_name = img_name[:-15] + '_Rp30_Annotation.png'
            img_Rm10_name = img_name[:-15] + '_Rm10_Annotation.png'
            img_Rm20_name = img_name[:-15] + '_Rm20_Annotation.png'
            img_Rm30_name = img_name[:-15] + '_Rm30_Annotation.png'

        else:
            print("Image augumentation falied! Please set the input parameter 'data' to 'image' or 'label'!")
            break

        # Save path
        img_path = out_folder + img_name
        img_085_path = out_folder + img_085_name
        img_115_path = out_folder + img_115_name
        img_Flr_path = out_folder + img_Flr_name
        img_Fud_path = out_folder + img_Fud_name
        img_Rp10_path = out_folder + img_Rp10_name
        img_Rp20_path = out_folder + img_Rp20_name
        img_Rp30_path = out_folder + img_Rp30_name
        img_Rm10_path = out_folder + img_Rm10_name
        img_Rm20_path = out_folder + img_Rm20_name
        img_Rm30_path = out_folder + img_Rm30_name

        # Save results
        cv2.imwrite(img_path, img)
        cv2.imwrite(img_085_path, img_085)
        cv2.imwrite(img_115_path, img_115)
        cv2.imwrite(img_Flr_path, img_Flr)
        cv2.imwrite(img_Fud_path, img_Fud)
        cv2.imwrite(img_Rp10_path, img_Rp10)
        cv2.imwrite(img_Rp20_path, img_Rp20)
        cv2.imwrite(img_Rp30_path, img_Rp30)
        cv2.imwrite(img_Rm10_path, img_Rm10)
        cv2.imwrite(img_Rm20_path, img_Rm20)
        cv2.imwrite(img_Rm30_path, img_Rm30)

    print("Images augumentation finished!")

In [5]:
HC18_training_set_folder = '/kaggle/input/fetal-head/training_set/training_set/'

# To save preprocess images
train_img_folder = '/kaggle/working/data/train/images/'
os.makedirs(train_img_folder)

# To save preprocess labels
train_label_folder = '/kaggle/working/data/train/labels/'
os.makedirs(train_label_folder)

In [6]:
dirs = os.listdir(HC18_training_set_folder)
for i in range(len(dirs)):
    print("Dividing images and labels: %d / %d" % (i+1,len(dirs)))
    img_name = dirs[i]
    img_path = HC18_training_set_folder + img_name
    img = cv2.imread(img_path,0)
    
    if img is None:
        print(f"Error: Image not loaded. Please check the image path: {img_path}")
        continue
    # Crop images
    crop_img = img_crop(img)

    # Save images
    if img_name.endswith('Annotation.png'):
        save_path = os.path.join(train_label_folder, img_name)
    else:
        save_path = os.path.join(train_img_folder, img_name)

    cv2.imwrite(save_path, crop_img)

Dividing images and labels: 1 / 1999
Dividing images and labels: 2 / 1999
Dividing images and labels: 3 / 1999
Dividing images and labels: 4 / 1999
Dividing images and labels: 5 / 1999
Dividing images and labels: 6 / 1999
Dividing images and labels: 7 / 1999
Dividing images and labels: 8 / 1999
Dividing images and labels: 9 / 1999
Dividing images and labels: 10 / 1999
Dividing images and labels: 11 / 1999
Dividing images and labels: 12 / 1999
Dividing images and labels: 13 / 1999
Dividing images and labels: 14 / 1999
Dividing images and labels: 15 / 1999
Dividing images and labels: 16 / 1999
Dividing images and labels: 17 / 1999
Dividing images and labels: 18 / 1999
Dividing images and labels: 19 / 1999
Dividing images and labels: 20 / 1999
Dividing images and labels: 21 / 1999
Dividing images and labels: 22 / 1999
Dividing images and labels: 23 / 1999
Dividing images and labels: 24 / 1999
Dividing images and labels: 25 / 1999
Dividing images and labels: 26 / 1999
Dividing images and l

In [7]:
ellip_fill(train_label_folder, train_label_folder)

Ellip filling: 1 / 999
Ellip filling: 2 / 999
Ellip filling: 3 / 999
Ellip filling: 4 / 999
Ellip filling: 5 / 999
Ellip filling: 6 / 999
Ellip filling: 7 / 999
Ellip filling: 8 / 999
Ellip filling: 9 / 999
Ellip filling: 10 / 999
Ellip filling: 11 / 999
Ellip filling: 12 / 999
Ellip filling: 13 / 999
Ellip filling: 14 / 999
Ellip filling: 15 / 999
Ellip filling: 16 / 999
Ellip filling: 17 / 999
Ellip filling: 18 / 999
Ellip filling: 19 / 999
Ellip filling: 20 / 999
Ellip filling: 21 / 999
Ellip filling: 22 / 999
Ellip filling: 23 / 999
Ellip filling: 24 / 999
Ellip filling: 25 / 999
Ellip filling: 26 / 999
Ellip filling: 27 / 999
Ellip filling: 28 / 999
Ellip filling: 29 / 999
Ellip filling: 30 / 999
Ellip filling: 31 / 999
Ellip filling: 32 / 999
Ellip filling: 33 / 999
Ellip filling: 34 / 999
Ellip filling: 35 / 999
Ellip filling: 36 / 999
Ellip filling: 37 / 999
Ellip filling: 38 / 999
Ellip filling: 39 / 999
Ellip filling: 40 / 999
Ellip filling: 41 / 999
Ellip filling: 42 / 999
E

In [8]:
val_img_folder = '/kaggle/working/data/validation/images/'
val_label_folder = '/kaggle/working/data/validation/labels/'
os.makedirs(val_img_folder)
os.makedirs(val_label_folder)

dirs = os.listdir(train_img_folder)
l = len(dirs)
val_l= round(0.2*l)
random.shuffle(dirs)

for i in range(val_l):
    print("Dividing the train and val set: %d / %d" % (i+1, val_l))
    img_name = dirs[i]
    label_name = img_name[:-4] + '_Annotation.png'

    img_path = train_img_folder + img_name
    label_path = train_label_folder + label_name

    img_save_path = val_img_folder + img_name
    label_save_path = val_label_folder + label_name

    img = cv2.imread(img_path, 0)
    label = cv2.imread(label_path, 0)

    cv2.imwrite(img_save_path, img)
    cv2.imwrite(label_save_path, label)

    # Remove validation images and labels from '../data/train/images' and '../data/train/labels'
    os.remove(img_path)
    os.remove(label_path)

Dividing the train and val set: 1 / 200
Dividing the train and val set: 2 / 200
Dividing the train and val set: 3 / 200
Dividing the train and val set: 4 / 200
Dividing the train and val set: 5 / 200
Dividing the train and val set: 6 / 200
Dividing the train and val set: 7 / 200
Dividing the train and val set: 8 / 200
Dividing the train and val set: 9 / 200
Dividing the train and val set: 10 / 200
Dividing the train and val set: 11 / 200
Dividing the train and val set: 12 / 200
Dividing the train and val set: 13 / 200
Dividing the train and val set: 14 / 200
Dividing the train and val set: 15 / 200
Dividing the train and val set: 16 / 200
Dividing the train and val set: 17 / 200
Dividing the train and val set: 18 / 200
Dividing the train and val set: 19 / 200
Dividing the train and val set: 20 / 200
Dividing the train and val set: 21 / 200
Dividing the train and val set: 22 / 200
Dividing the train and val set: 23 / 200
Dividing the train and val set: 24 / 200
Dividing the train and va

In [9]:
pixel_size_hc_file = '/kaggle/input/fetal-head/training_set_pixel_size_and_HC.csv'
val_pixel_size_hc_save = '/kaggle/working/data/validation/val_set_pixel_size_and_HC.csv'
p_size_hc = pd.read_csv(pixel_size_hc_file)

fn = p_size_hc['filename'].values
ps = p_size_hc['pixel size'].values
hc = p_size_hc['head circumference (mm)'].values

new_psize_hc = pd.DataFrame({'pixel size':ps,'head circumference (mm)':hc},index=fn)

val_psize_hc = pd.DataFrame(new_psize_hc,index=dirs[0:val_l])

val_fn = val_psize_hc.index
val_ps = val_psize_hc['pixel size'].values
val_hc = val_psize_hc['head circumference (mm)'].values

val_psize_hc = pd.DataFrame({'filename':val_fn, 'pixel size':val_ps, 'head circumference (mm)':val_hc})
val_psize_hc.to_csv(val_pixel_size_hc_save,index=False)

In [10]:
# Data augumentation in train set
train_augu_img_folder = '/kaggle/working/data/train/augu_images/'
train_augu_label_folder = '/kaggle/working/data/train/augu_labels/'
os.makedirs(train_augu_img_folder)
os.makedirs(train_augu_label_folder)

In [11]:
# Image augumentation
img_augumentation(train_img_folder,train_augu_img_folder,data="image")

image augumentation: 1 / 799
image augumentation: 2 / 799
image augumentation: 3 / 799
image augumentation: 4 / 799
image augumentation: 5 / 799
image augumentation: 6 / 799
image augumentation: 7 / 799
image augumentation: 8 / 799
image augumentation: 9 / 799
image augumentation: 10 / 799
image augumentation: 11 / 799
image augumentation: 12 / 799
image augumentation: 13 / 799
image augumentation: 14 / 799
image augumentation: 15 / 799
image augumentation: 16 / 799
image augumentation: 17 / 799
image augumentation: 18 / 799
image augumentation: 19 / 799
image augumentation: 20 / 799
image augumentation: 21 / 799
image augumentation: 22 / 799
image augumentation: 23 / 799
image augumentation: 24 / 799
image augumentation: 25 / 799
image augumentation: 26 / 799
image augumentation: 27 / 799
image augumentation: 28 / 799
image augumentation: 29 / 799
image augumentation: 30 / 799
image augumentation: 31 / 799
image augumentation: 32 / 799
image augumentation: 33 / 799
image augumentation

In [12]:
# Label augumentation
img_augumentation(train_label_folder, train_augu_label_folder, data="label")

label augumentation: 1 / 799
label augumentation: 2 / 799
label augumentation: 3 / 799
label augumentation: 4 / 799
label augumentation: 5 / 799
label augumentation: 6 / 799
label augumentation: 7 / 799
label augumentation: 8 / 799
label augumentation: 9 / 799
label augumentation: 10 / 799
label augumentation: 11 / 799
label augumentation: 12 / 799
label augumentation: 13 / 799
label augumentation: 14 / 799
label augumentation: 15 / 799
label augumentation: 16 / 799
label augumentation: 17 / 799
label augumentation: 18 / 799
label augumentation: 19 / 799
label augumentation: 20 / 799
label augumentation: 21 / 799
label augumentation: 22 / 799
label augumentation: 23 / 799
label augumentation: 24 / 799
label augumentation: 25 / 799
label augumentation: 26 / 799
label augumentation: 27 / 799
label augumentation: 28 / 799
label augumentation: 29 / 799
label augumentation: 30 / 799
label augumentation: 31 / 799
label augumentation: 32 / 799
label augumentation: 33 / 799
label augumentation

# TRAINING THE MODEL

The model is designed for image segmentation, which involves partitioning an image into meaningful segments, such as identifying different objects or regions within an image. The CSM class uses a multi-stage approach to refine the segmentation masks progressively. The network's architecture, with multiple stages and feature concatenations, allows it to capture various levels of detail and improve the segmentation results iteratively.

In [13]:
class HcDataset(Dataset):
    """
    Create dataset for model training
    """

    def __init__(self, input_folder, label_folder, input_size=[192, 128]):

        self.input_w = int(input_size[0])
        self.input_h = int(input_size[1])

        self.label1_w = int(self.input_w / 8)
        self.label1_h = int(self.input_h / 8)

        self.label2_w = int(self.input_w / 4)
        self.label2_h = int(self.input_h / 4)

        self.input_list = os.listdir(input_folder)

        self.input_folder = input_folder
        self.label_folder = label_folder

        self.dataset_L = len(self.input_list)

    def __len__(self):
        return self.dataset_L

    def __getitem__(self, idx):

        input_name = self.input_list[idx]
        label_name = input_name[:-4] + '_Annotation.png'

        input_file_path = self.input_folder + input_name
        label_file_path = self.label_folder + label_name

        img = cv2.imread(input_file_path,0)
        label = cv2.imread(label_file_path,0)

        # Resize images and labels
        img = cv2.resize(img, (self.input_w, self.input_h), interpolation=cv2.INTER_AREA)
        label1 = cv2.resize(label,(self.label1_w, self.label1_h),interpolation=cv2.INTER_AREA)
        label2 = cv2.resize(label,(self.label2_w, self.label2_h),interpolation=cv2.INTER_AREA)

        img = img / 255
        label1 = label1 / 255
        label2 = label2 / 255

        img = img.astype('float32')
        label1 = label1.astype('float32')
        label2 = label2.astype('float32')

        input = torch.from_numpy(img)
        label1 = torch.from_numpy(label1)
        label2 = torch.from_numpy(label2)

        input = input.unsqueeze(0)
        label1 = label1.unsqueeze(0)
        label2 = label2.unsqueeze(0)

        return input, label1,label2

In [14]:
class CSM(nn.Module):
    """
    Convolutional segmentation machine.
    """

    def __init__(self):
        super(CSM,self).__init__()
        self.stage1 = nn.Sequential(nn.Conv2d(1, 8, 9, padding=4),
                                   nn.BatchNorm2d(8),
                                   nn.ReLU(True),
                                   nn.MaxPool2d(2),
                                   nn.Conv2d(8, 8, 9, padding=4, groups=8),
                                   nn.Conv2d(8, 16, 1),
                                   nn.BatchNorm2d(16),
                                   nn.ReLU(True),
                                   nn.MaxPool2d(2),
                                   nn.Conv2d(16, 16, 9, padding=4, groups=16),
                                   nn.Conv2d(16, 32, 1),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(True),
                                   nn.MaxPool2d(2),
                                   nn.Conv2d(32, 32, 5, padding=2, groups=32),
                                   nn.Conv2d(32, 64, 1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(True),
                                   nn.Conv2d(64, 64, 9, padding=4, groups=64),
                                   nn.Conv2d(64, 32, 1),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(True),
                                   nn.Conv2d(32, 16, 1),
                                   nn.ReLU(True),
                                   nn.Conv2d(16, 1, 1),
                                   nn.Sigmoid())

        self.f1 = nn.Sequential(nn.Conv2d(1, 8, 9, padding=4),
                                nn.BatchNorm2d(8),
                                nn.ReLU(True),
                                nn.MaxPool2d(2),
                                nn.Conv2d(8, 16, 9, padding=4),
                                nn.BatchNorm2d(16),
                                nn.ReLU(True),
                                nn.MaxPool2d(2),
                                nn.Conv2d(16, 32, 9, padding=4),
                                nn.BatchNorm2d(32),
                                nn.ReLU(True))

        self.f2 = nn.Sequential(nn.MaxPool2d(2),
                                  nn.Conv2d(32, 16, 5, padding=2),
                                  nn.BatchNorm2d(16),
                                  nn.ReLU(True))

        self.up = nn.UpsamplingBilinear2d(scale_factor=2)

        self.f3 = nn.Sequential(nn.Conv2d(32, 16, 3, padding=1),
                                  nn.BatchNorm2d(16),
                                  nn.ReLU(True),
                                  nn.Conv2d(16, 16, 3, padding=1),
                                  nn.BatchNorm2d(16),
                                  nn.ReLU(True))

        self.stage2 = CSM_stagen()
        self.stage3 = CSM_stagen()

    def forward(self,x):
        y1 = self.stage1(x)
        x_f1 = self.f1(x)

        x_f2 = self.f2(x_f1)
        x_f3 = self.f3(x_f1)

        x1 = torch.cat([y1,x_f2],1)
        y2 = self.stage2(x1)
        y2_up = self.up(y2)
        x2 = torch.cat([y2_up,x_f3],1)
        y3 = self.stage3(x2)

        return y1, y2, y3

In [15]:
class CSM_stagen(nn.Module):
    """
    Network of n(n>=2) stage in CSM.
    """
    def __init__(self):
        super(CSM_stagen,self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(17, 17, 11, padding=5,groups=17),
                                  nn.Conv2d(17, 32, 1),
                                  nn.BatchNorm2d(32),
                                  nn.ReLU(True),
                                  nn.Conv2d(32, 32, 11, padding=5, groups=32),
                                  nn.Conv2d(32, 64, 1),
                                  nn.BatchNorm2d(64),
                                  nn.ReLU(True),
                                  nn.Conv2d(64, 64, 11, padding=5, groups=64),
                                  nn.Conv2d(64, 32, 1),
                                  nn.BatchNorm2d(32),
                                  nn.ReLU(True),
                                  nn.Conv2d(32, 16, 1),
                                  nn.BatchNorm2d(16),
                                  nn.ReLU(True),
                                  nn.Conv2d(16, 1, 1),
                                  nn.Sigmoid())

    def forward(self,x):
        x = self.conv(x)

        return x

In [20]:
def train_model(model, dataloader, epoches=20, lr=0.001, device='cuda', save_model_name='models/trained_model.pth'):
    """
    Train the deep model and save result.
    """

    model.to(device)
    print("Using {} device".format(device))

    # Loss function and optimization methods
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train
    time_start = time.time()
    for epoch in range(epoches):
        running_loss = 0.0
        for data in dataloader:
            inputs, label1, label2 = data[0].to(device), data[1].to(device), data[2].to(device)
            optimizer.zero_grad()
            pre1, pre2, outputs = model(inputs)

            loss1 = criterion(pre1, label1)
            loss2 = criterion(pre2, label1)
            loss3 = criterion(outputs, label2)
            loss = loss1 + loss2 + loss3
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print("This is epoch %d / %d" % ((epoch + 1), epoches))
        print(" running loss : %f" % running_loss)
    print("Finished training !")

    # Save model
    if save_model_name[-4:] != '.pth':
        save_model_name = save_model_name + '.pth'
    torch.save(model.state_dict(), save_model_name)
    time_end = time.time()
    print("Time cost: %f s" % (time_end - time_start))

In [16]:
from torch.utils.data import DataLoader

input_folder = '/kaggle/working/data/train/augu_images/'
label_folder = '/kaggle/working/data/train/augu_labels/'

#  Training data
train_data = HcDataset(input_folder,label_folder,[192,128])
dataloader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)

In [17]:
# CSM model
net = CSM()

In [22]:
# File to save the trained model
os.makedirs('/kaggle/working/models/')
save_model_name='/kaggle/working/models/test_model.pth'

In [23]:
# Model training
train_model(model=net,dataloader=dataloader,epoches=20,lr=0.001,device='cuda',save_model_name=save_model_name)

Using cuda device
This is epoch 1 / 20
 running loss : 3.253197
This is epoch 2 / 20
 running loss : 3.101781
This is epoch 3 / 20
 running loss : 2.813345
This is epoch 4 / 20
 running loss : 2.736699
This is epoch 5 / 20
 running loss : 2.686211
This is epoch 6 / 20
 running loss : 2.632642
This is epoch 7 / 20
 running loss : 2.473922
This is epoch 8 / 20
 running loss : 2.511789
This is epoch 9 / 20
 running loss : 2.361507
This is epoch 10 / 20
 running loss : 2.277908
This is epoch 11 / 20
 running loss : 2.243339
This is epoch 12 / 20
 running loss : 2.195132
This is epoch 13 / 20
 running loss : 2.155188
This is epoch 14 / 20
 running loss : 2.075538
This is epoch 15 / 20
 running loss : 2.099909
This is epoch 16 / 20
 running loss : 2.061723
This is epoch 17 / 20
 running loss : 1.975272
This is epoch 18 / 20
 running loss : 1.970295
This is epoch 19 / 20
 running loss : 1.906996
This is epoch 20 / 20
 running loss : 1.855852
Finished training !
Time cost: 1302.101149 s


# PREDICTION 

In [24]:
def predict(model, input_folder, predict_folder,device="cpu"):
    """
    Predict using the trained model and save results.
    """

    model.to(device)
    print("Using {} device".format(device))
    model.eval()

    # Resize factor
    k1 = 4
    w = int(768 / k1)
    h = int(512 / k1)

    # Predict
    dirs = os.listdir(input_folder)
    time_start = time.time()
    for i in range(len(dirs)):
        print('Predicting: Image = %d / %d' % (i + 1, len(dirs)))
        img_name = dirs[i]
        predict_name = img_name
        img_path = input_folder + img_name

        img = cv2.imread(img_path, 0)

        # Resize images
        input_img = cv2.resize(img, (w, h), interpolation=cv2.INTER_AREA)
        input_img = input_img.astype('float32')
        input_img = input_img / 255.0

        # To torch.tensor
        input_img = torch.from_numpy(input_img)
        input_img = input_img.unsqueeze(0)
        input_img = input_img.unsqueeze(0)

        input_img = input_img.to(device)

        # Predcit
        _, _, predict = model(input_img)
        predict = predict[0, 0, :, :]
        predict = predict.cpu().detach().numpy()
        predict = np.round(predict)

        # Save
        predict = predict * 255
        predict = predict.astype('uint8')

        predict_path = predict_folder + predict_name

        cv2.imwrite(predict_path, predict)

    # Time consumed
    time_end = time.time()
    print('Finish prediction!')
    print('Time consume: %f' % (time_end - time_start))

In [25]:
input_folder = '/kaggle/working/data/validation/images/'

# Prediction folder for results saving
predict_folder = '/kaggle/working/results/predictions/'
if os.path.isdir(predict_folder):
    shutil.rmtree(predict_folder)
os.makedirs(predict_folder)

# Load the network
net_dict_file = '/kaggle/working/models/test_model.pth'
net = CSM()
net.load_state_dict(torch.load(net_dict_file))

# Predict
predict(net,input_folder,predict_folder,device='cuda')


Using cuda device
Predicting: Image = 1 / 200
Predicting: Image = 2 / 200
Predicting: Image = 3 / 200
Predicting: Image = 4 / 200
Predicting: Image = 5 / 200
Predicting: Image = 6 / 200
Predicting: Image = 7 / 200
Predicting: Image = 8 / 200
Predicting: Image = 9 / 200
Predicting: Image = 10 / 200
Predicting: Image = 11 / 200
Predicting: Image = 12 / 200
Predicting: Image = 13 / 200
Predicting: Image = 14 / 200
Predicting: Image = 15 / 200
Predicting: Image = 16 / 200
Predicting: Image = 17 / 200
Predicting: Image = 18 / 200
Predicting: Image = 19 / 200
Predicting: Image = 20 / 200
Predicting: Image = 21 / 200
Predicting: Image = 22 / 200
Predicting: Image = 23 / 200
Predicting: Image = 24 / 200
Predicting: Image = 25 / 200
Predicting: Image = 26 / 200
Predicting: Image = 27 / 200
Predicting: Image = 28 / 200
Predicting: Image = 29 / 200
Predicting: Image = 30 / 200
Predicting: Image = 31 / 200
Predicting: Image = 32 / 200
Predicting: Image = 33 / 200
Predicting: Image = 34 / 200
Predi

# POSTPROCESSING

In [26]:
def mcc_edge(in_img):
    """
    Extract max connected component and then extract edge.
    """

    img = in_img
    if in_img.dtype != 'uint8':
        in_img = in_img * 255
        img = in_img.astype('uint8')

    # Max connected component extraction
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
    sort_label = np.argsort(-stats[:, 4], )

    idx = labels == sort_label[1]
    max_connect = idx * 255
    max_connect = max_connect.astype('uint8')

    # Edge detection
    edge_img = cv2.Canny(max_connect, 50, 250)

    return edge_img

In [27]:
def ellip_fit(in_img):
    """
    To fit the fetal head contour into an ellipse, output 5 ellipse parameters.
    Note: The unit of parameters obtained is pixel distance, and the coordinate system is as follows:
    the upper left corner of the image is the origin,vertically downward is the X-axis direction,
    horizontally right is the Y-axis direction.
    """

    edge_img = in_img
    if in_img.dtype != 'uint8':
        in_img = in_img * 255
        edge_img = in_img.astype('uint8')

    # Get coordinates of edge points
    edge_points = np.where(edge_img == 255)
    edge_x = edge_points[0]
    edge_y = edge_points[1]
    edge_x = edge_x.reshape(-1, 1)  # (N, 1)
    edge_y = edge_y.reshape(-1, 1)  # (N, 1)

    # least squares fitting
    x2 = edge_x * edge_x
    xy = 2 * edge_x * edge_y
    _2x = -2 * edge_x
    _2y = -2 * edge_y
    mine_1 = -np.ones(edge_x.shape)
    X = np.concatenate((x2, xy, _2x, _2y, mine_1), axis=1)
    y = -edge_y * edge_y


    model = LinearRegression(fit_intercept=False)
    model.fit(X, y)
    k1 = model.coef_[0, 0]
    k2 = model.coef_[0, 1]
    k3 = model.coef_[0, 2]
    k4 = model.coef_[0, 3]
    k5 = model.coef_[0, 4]

    # Calculate parameters: xc,yc,theta,a,b
    xc = (k3 - k2 * k4) / (k1 - k2 * k2)
    yc = (k1 * k4 - k2 * k3) / (k1 - k2 * k2)
    theta = 0.5 * np.arctan(2 * k2 / (k1 - 1))


    T = np.tan(theta)
    K = (1 - k1 * T * T) / (k1 - T * T)                         # a^2 = K * b^2
    p1 = -np.square(xc + T * yc)
    p2 = -np.square(xc * T - yc)
    b_2 = (k5 * (T * T + K) - p1 - p2 * K) / (K * (T * T + 1))  # b^2
    a_2 = K * b_2                                               # a^2

    a = np.sqrt(a_2)
    b = np.sqrt(b_2)

    # Set a to the long half axis and b to the short half axis, and adjust the angle
    if a < b:
        t = b
        b = a
        a = t
        theta = theta + 0.5 * np.pi

    return xc,yc,theta,a,b

In [28]:
# Folder of model predictions
input_folder = '/kaggle/working/results/predictions/'

#  Folder to save postprocess results
edge_folder = '/kaggle/working/results/predictions_edge/'
if os.path.isdir(edge_folder):
    shutil.rmtree(edge_folder)
os.makedirs(edge_folder)

# Extract fetal contour
dirs = os.listdir(input_folder)
for i in range(len(dirs)):
    print('Extracting max connect component edge: Image = %d / %d' % (i + 1, len(dirs)))
    img_name = dirs[i]
    img_path = input_folder + img_name

    img = cv2.imread(img_path, 0)
    edge_img = mcc_edge(img)

    save_path = edge_folder + img_name
    cv2.imwrite(save_path, edge_img)

Extracting max connect component edge: Image = 1 / 200
Extracting max connect component edge: Image = 2 / 200
Extracting max connect component edge: Image = 3 / 200
Extracting max connect component edge: Image = 4 / 200
Extracting max connect component edge: Image = 5 / 200
Extracting max connect component edge: Image = 6 / 200
Extracting max connect component edge: Image = 7 / 200
Extracting max connect component edge: Image = 8 / 200
Extracting max connect component edge: Image = 9 / 200
Extracting max connect component edge: Image = 10 / 200
Extracting max connect component edge: Image = 11 / 200
Extracting max connect component edge: Image = 12 / 200
Extracting max connect component edge: Image = 13 / 200
Extracting max connect component edge: Image = 14 / 200
Extracting max connect component edge: Image = 15 / 200
Extracting max connect component edge: Image = 16 / 200
Extracting max connect component edge: Image = 17 / 200
Extracting max connect component edge: Image = 18 / 200
E

# VISUALISATION

In [30]:
# Visualization
def draw_ellip(xc,yc,a,b,theta,in_img,color='r'):
    """
    Image visualization by drawing predicted ellipse on original ultrasound image.
    """

    if len(in_img.shape) == 3:
        img = in_img
    else:
        w,h = in_img.shape
        img = np.zeros([w,h,3],in_img.dtype)
        img[:,:,0] = in_img
        img[:,:,1] = in_img
        img[:,:,2] = in_img

    if img.dtype != 'uint8':
        img = img*255
        img = img.astype('uint8')

    # Calculate the elliptic coordinate points
    rng = np.arange(0, 2 * np.pi, 0.001)
    ellipse_x = a * np.sin(rng)
    ellipse_y = b * np.cos(rng)

    # Rotate
    theta = theta
    ellipse_x1 = ellipse_x * np.cos(theta) - ellipse_y * np.sin(theta)
    ellipse_y1 = ellipse_x * np.sin(theta) + ellipse_y * np.cos(theta)

    # Translation
    ellipse_x1 = ellipse_x1 + xc
    ellipse_y1 = ellipse_y1 + yc

    # Round
    ellipse_x1 = ellipse_x1.astype(int)
    ellipse_y1 = ellipse_y1.astype(int)

    ellipse_x1 = ellipse_x1 % w
    ellipse_y1 = ellipse_y1 % h

    # Draw ellipse
    img[ellipse_x1, ellipse_y1, 0] = 0
    img[ellipse_x1, ellipse_y1, 1] = 0
    img[ellipse_x1, ellipse_y1, 2] = 0
    if color=='b':
        img[ellipse_x1,ellipse_y1,0]=255
    elif color=='g':
        img[ellipse_x1,ellipse_y1,1]=255
    elif color=='r':
        img[ellipse_x1,ellipse_y1,2]=255
    else:
        img[ellipse_x1,ellipse_y1,0] = 255
        img[ellipse_x1, ellipse_y1, 1] = 255
        img[ellipse_x1, ellipse_y1, 2] = 255

    return img

In [31]:
# Postprocess results folder
edge_folder = '/kaggle/working/results/predictions_edge/'

# To save ellipse parameters
results = []
name = ['filename', 'center_x(pixel)', 'center_y(pixel)', 'semi_axes_a(pixel)',
        'semi_axes_b(pixel)', 'HC(pixel)', 'angle(rad)']

# Filename of ellipse parameters file
save_ellip_para_file = '/kaggle/working/results/ellip_params.csv'

# upsample factor
u = 16

# Ellipse fitting to obtain parameters
dirs = os.listdir(edge_folder)
for i in range(len(dirs)):
    print('Ellip fitting: Image = %d / %d' % (i + 1, len(dirs)))
    img_name = dirs[i]
    img_path = edge_folder + img_name

    edge_img = cv2.imread(img_path, 0)
    xc, yc, theta, a, b = ellip_fit(edge_img)

    # Restore to original size with the factor 'u'
    xc = (xc + 0.5) * u - 0.5
    yc = (yc + 0.5) * u - 0.5
    a = a * u
    b = b * u
    hc = 2 * np.pi * b + 4 * (a - b)  # HC

    results.append([img_name, xc, yc, a, b, hc, theta])

# Save
predict_results = pd.DataFrame(columns=name, data=results)
predict_results.to_csv(save_ellip_para_file, index=False)

Ellip fitting: Image = 1 / 200
Ellip fitting: Image = 2 / 200
Ellip fitting: Image = 3 / 200
Ellip fitting: Image = 4 / 200
Ellip fitting: Image = 5 / 200
Ellip fitting: Image = 6 / 200
Ellip fitting: Image = 7 / 200
Ellip fitting: Image = 8 / 200
Ellip fitting: Image = 9 / 200
Ellip fitting: Image = 10 / 200
Ellip fitting: Image = 11 / 200
Ellip fitting: Image = 12 / 200
Ellip fitting: Image = 13 / 200
Ellip fitting: Image = 14 / 200
Ellip fitting: Image = 15 / 200
Ellip fitting: Image = 16 / 200
Ellip fitting: Image = 17 / 200
Ellip fitting: Image = 18 / 200
Ellip fitting: Image = 19 / 200
Ellip fitting: Image = 20 / 200
Ellip fitting: Image = 21 / 200
Ellip fitting: Image = 22 / 200
Ellip fitting: Image = 23 / 200
Ellip fitting: Image = 24 / 200
Ellip fitting: Image = 25 / 200
Ellip fitting: Image = 26 / 200
Ellip fitting: Image = 27 / 200
Ellip fitting: Image = 28 / 200
Ellip fitting: Image = 29 / 200
Ellip fitting: Image = 30 / 200
Ellip fitting: Image = 31 / 200
Ellip fitting: Im

In [32]:
# Validation images folder
img_folder = '/kaggle/working/data/validation/images/'

# Folder to save visualization results
visual_folder = '/kaggle/working/results/visualizations/'
if os.path.isdir(visual_folder):
    shutil.rmtree(visual_folder)
os.makedirs(visual_folder)

# Prediction ellipse parameters file
predict_ellip_params_file = '/kaggle/working/results/ellip_params.csv'

params_data = pd.read_csv(predict_ellip_params_file)
v = params_data.values

for i in range(len(params_data)):
    print("Image processing: %d / %d" % (i+1, len(params_data)))
    img_name = v[i, 0]
    xc = v[i, 1]
    yc = v[i, 2]
    a = v[i, 3]
    b = v[i, 4]
    theta = v[i, 6]

    img_path = img_folder + img_name
    img = cv2.imread(img_path,0)

    # Draw ellipse on the original image using predicted ellipse parameters
    visual_img = draw_ellip(xc, yc, a, b, theta, img, 'r')

    # Save
    visual_path = visual_folder + img_name
    cv2.imwrite(visual_path, visual_img)

Image processing: 1 / 200
Image processing: 2 / 200
Image processing: 3 / 200
Image processing: 4 / 200
Image processing: 5 / 200
Image processing: 6 / 200
Image processing: 7 / 200
Image processing: 8 / 200
Image processing: 9 / 200
Image processing: 10 / 200
Image processing: 11 / 200
Image processing: 12 / 200
Image processing: 13 / 200
Image processing: 14 / 200
Image processing: 15 / 200
Image processing: 16 / 200
Image processing: 17 / 200
Image processing: 18 / 200
Image processing: 19 / 200
Image processing: 20 / 200
Image processing: 21 / 200
Image processing: 22 / 200
Image processing: 23 / 200
Image processing: 24 / 200
Image processing: 25 / 200
Image processing: 26 / 200
Image processing: 27 / 200
Image processing: 28 / 200
Image processing: 29 / 200
Image processing: 30 / 200
Image processing: 31 / 200
Image processing: 32 / 200
Image processing: 33 / 200
Image processing: 34 / 200
Image processing: 35 / 200
Image processing: 36 / 200
Image processing: 37 / 200
Image proc

# EVALUATION

In [73]:
def dice(img1, img2):
    """
    Calculate dice coefficient between two images.
    """
    img1 = np.round(img1 / 255).astype('uint8')
    img2 = np.round(img2 / 255).astype('uint8')
    s1 = np.sum(img1)
    s2 = np.sum(img2)
    s = np.sum(cv2.bitwise_and(img1, img2))
    d = 2 * s / (s1 + s2)
    return d

In [74]:
def dice(img1, img2):
    """
    Calculate dice coefficient between two images.
    """
    img1 = np.round(img1 / 255).astype('uint8')
    img2 = np.round(img2 / 255).astype('uint8')
    s1 = np.sum(img1)
    s2 = np.sum(img2)
    s = np.sum(cv2.bitwise_and(img1, img2))
    d = 2 * s / (s1 + s2)
    return d

def dice_folder(label_folder, predict_folder):
    """
    Calculate dice coefficient in between tow folder.
    """
    dirs = os.listdir(predict_folder)
    D = []
    print('Calculating dice coefficient ...')
    for i in range(len(dirs)):
        predict_name = dirs[i]
        predict_path = predict_folder + predict_name

        label_name = predict_name[:-4] + '_Annotation.png'
        label_path = label_folder + label_name

        label = cv2.imread(label_path, 0)
        predict = cv2.imread(predict_path,0)

        r,c = label.shape
        resize_predict = cv2.resize(predict, (c, r), interpolation=cv2.INTER_CUBIC)
        resize_predict = (np.round(resize_predict / 255) * 255).astype('uint8')

        d = dice(label, resize_predict)
        D.append(d)

    dice_series = pd.Series(D, index=dirs)
    return dice_series

In [75]:
img_folder = '/kaggle/working/data/validation/images/'

# Folder to save visualization results
visual_folder = '/kaggle/working/results/visualizations/'
if os.path.isdir(visual_folder):
    shutil.rmtree(visual_folder)
os.makedirs(visual_folder)

# Prediction ellipse parameters file
predict_ellip_params_file = '/kaggle/working/results/ellip_params.csv'

params_data = pd.read_csv(predict_ellip_params_file)
v = params_data.values

for i in range(len(params_data)):
    print("Image processing: %d / %d" % (i+1, len(params_data)))
    img_name = v[i, 0]
    xc = v[i, 1]
    yc = v[i, 2]
    a = v[i, 3]
    b = v[i, 4]
    theta = v[i, 6]

    img_path = img_folder + img_name
    img = cv2.imread(img_path,0)

    # Draw ellipse on the original image using predicted ellipse parameters
    visual_img = draw_ellip(xc, yc, a, b, theta, img, 'r')

    # Save
    visual_path = visual_folder + img_name
    cv2.imwrite(visual_path, visual_img)

Image processing: 1 / 200
Image processing: 2 / 200
Image processing: 3 / 200
Image processing: 4 / 200
Image processing: 5 / 200
Image processing: 6 / 200
Image processing: 7 / 200
Image processing: 8 / 200
Image processing: 9 / 200
Image processing: 10 / 200
Image processing: 11 / 200
Image processing: 12 / 200
Image processing: 13 / 200
Image processing: 14 / 200
Image processing: 15 / 200
Image processing: 16 / 200
Image processing: 17 / 200
Image processing: 18 / 200
Image processing: 19 / 200
Image processing: 20 / 200
Image processing: 21 / 200
Image processing: 22 / 200
Image processing: 23 / 200
Image processing: 24 / 200
Image processing: 25 / 200
Image processing: 26 / 200
Image processing: 27 / 200
Image processing: 28 / 200
Image processing: 29 / 200
Image processing: 30 / 200
Image processing: 31 / 200
Image processing: 32 / 200
Image processing: 33 / 200
Image processing: 34 / 200
Image processing: 35 / 200
Image processing: 36 / 200
Image processing: 37 / 200
Image proc

In [76]:
params_label_file = '/kaggle/working/data/validation/val_set_pixel_size_and_HC.csv'
params_predict_file = '/kaggle/working/results/ellip_params.csv'

In [77]:
# Calculate mean difference and mean abs difference
params_label = pd.read_csv(params_label_file)
params_predict = pd.read_csv(params_predict_file)

label_fn = params_label['filename'].values
label_ps = params_label['pixel size'].values
label_hc = params_label['head circumference (mm)'].values

predict_fn = params_predict['filename'].values
predict_hc = params_predict['HC(pixel)'].values

label_ps_series = pd.Series(label_ps,label_fn)
label_hc_series = pd.Series(label_hc,label_fn)


predict_hc_series = pd.Series(predict_hc,predict_fn)


eval_results = pd.DataFrame({'pixel size(mm)':label_ps_series, 'HC_truth(mm)':label_hc_series,
                             'HC_predict(pixel)': predict_hc_series})

eval_results['HC_predict(mm)'] = eval_results['HC_predict(pixel)'] * eval_results['pixel size(mm)']
eval_results['Difference(mm)'] = eval_results['HC_predict(mm)'] - eval_results['HC_truth(mm)']
eval_results['Abs_Difference(mm)'] = abs(eval_results['Difference(mm)'])

ave_adf = np.average(eval_results['Abs_Difference(mm)'].values)
ave_df = np.average(eval_results['Difference(mm)'].values)

print("Mean difference: %f(mm)" % (ave_df))
print("Mean absolulate difference: %f(mm)" % (ave_adf))

Mean difference: 3.702889(mm)
Mean absolulate difference: 4.209150(mm)


In [78]:
label_folder = '/kaggle/working/data/validation/labels/'
predict_folder = '/kaggle/working/results/predictions/'

In [79]:
# Calculate mean dice cofficience
dice = dice_folder(label_folder,predict_folder)
eval_results['Dice'] = dice
ave_dice = np.average(dice.values)
print('Mean dice: %f' % ave_dice)

Calculating dice coefficient ...
Mean dice: 0.970637


In [82]:
import os
import numpy as np
import pandas as pd
import cv2
from scipy.spatial.distance import directed_hausdorff

def hausdorff_folder(label_folder, predict_folder, pixel_size):
    """
    Calculate Hausdorff distance between images in two folders.
    """
    dirs = os.listdir(predict_folder)
    D = []
    print('Calculating Hausdorff distance ...')
    for predict_name in dirs:
        predict_path = os.path.join(predict_folder, predict_name)

        label_name = predict_name[:-4] + '_Annotation.png'
        label_path = os.path.join(label_folder, label_name)

        label = cv2.imread(label_path, 0)
        predict = cv2.imread(predict_path, 0)

        if label is None or predict is None:
            print(f"Error reading {predict_name} or corresponding label.")
            continue

        r, c = label.shape
        resize_predict = cv2.resize(predict, (c, r), interpolation=cv2.INTER_CUBIC)
        resize_predict = (np.round(resize_predict / 255) * 255).astype('uint8')

        d = hausdorff_d(label, resize_predict) * pixel_size[predict_name]
        D.append(d)

    hausdorff_d_series = pd.Series(D, index=dirs)
    return hausdorff_d_series

def hausdorff_d(img1, img2):
    """
    Calculate Hausdorff distance between two images using directed Hausdorff distance.
    """
    def extract_contour_points(img):
        contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        if len(contours) == 0:
            return np.empty((0, 2))
        return np.vstack([contour.squeeze() for contour in contours])

    points1 = extract_contour_points(img1)
    points2 = extract_contour_points(img2)

    d1 = directed_hausdorff(points1, points2)[0]
    d2 = directed_hausdorff(points2, points1)[0]

    return max(d1, d2)

hd = hausdorff_folder(label_folder, predict_folder, label_ps_series)
eval_results = {}
eval_results['Hausdorff distance(mm)'] = hd
ave_hd = np.average(hd.values)
print('Mean Hausdorff distance between predict and label is: %f(mm)' % ave_hd)


Calculating Hausdorff distance ...
Mean Hausdorff distance between predict and label is: 2.312199(mm)


In [86]:
import pandas as pd

# Convert the dictionary to a DataFrame
eval_results_df = pd.DataFrame(list(eval_results.items()), columns=['Metric', 'Value'])

# Save the DataFrame to a CSV file
save_path = '/kaggle/working/results/eval_results.csv'
eval_results_df.to_csv(save_path, index=False)

print(f'Evaluation results saved to {save_path}')


Evaluation results saved to /kaggle/working/results/eval_results.csv


# INTERPRETATION

- Hausdorff Distance:
Mean Hausdorff distance between predict and label is: 2.312199 mm

Interpretation: The Hausdorff distance measures the maximum distance between the predicted segmentation boundary and the ground truth boundary. A mean Hausdorff distance of 2.312199 mm indicates that, on average, the furthest point of the predicted segmentation is 2.312199 mm away from the nearest point on the ground truth segmentation. This low value suggests that the predicted segmentations are generally close to the true segmentations.

- Dice Coefficient:
Mean dice: 0.970637

Interpretation: The Dice coefficient is a measure of overlap between the predicted segmentation and the ground truth segmentation. It ranges from 0 to 1, where 1 indicates perfect overlap. A mean Dice coefficient of 0.970637 indicates a very high degree of overlap, suggesting that the predicted segmentations are highly accurate and closely match the ground truth.

- Mean Difference:
Mean difference: 3.702889 mm

Interpretation: The mean difference measures the average difference between the predicted and ground truth segmentations. A mean difference of 3.702889 mm suggests that, on average, the predictions differ from the ground truth by 3.702889 mm. While the Hausdorff distance captures the worst-case scenario, the mean difference provides an average measure of accuracy across all points.

- Mean Absolute Difference:
Mean absolute difference: 4.209150 mm

Interpretation: The mean absolute difference is similar to the mean difference but takes the absolute value of differences, ensuring that all deviations are treated as positive. This metric gives a sense of the average magnitude of errors without considering the direction of the errors. A mean absolute difference of 4.209150 mm indicates that the average size of deviations, irrespective of direction, is 4.209150 mm.

# Overall Interpretation

The evaluation metrics indicate that the model's predictions are very close to the ground truth segmentations:

- The low Hausdorff distance shows that the maximum error in boundary prediction is small.

- The high Dice coefficient indicates a high overlap between predicted and true segmentations.

- The mean difference and mean absolute difference are relatively low, suggesting that the average errors in the predictions are small.